In [ ]:
# Install required libraries
!pip install pandas numpy matplotlib plotly rasterio

In [ ]:
import os  # Module for interacting with the operating system
import pandas as pd  # Library for data manipulation and analysis
from xml.etree import ElementTree as ET  # Module for parsing and creating XML data

# Define the path to the extracted KML file
kml_file_path = os.path.join("C:/Users/subha/Desktop/DEM/Rudraprayag_DEM", "Rudraprayag.kml")

# Function to parse the KML file and extract coordinates
def extract_coordinates_from_kml(kml_file):
    try:
        # Parse the KML file and create an element tree object
        tree = ET.parse(kml_file)
        root = tree.getroot()  # Get the root element of the KML file

        # Namespace for KML files (used for finding elements with namespaces)
        namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

        # Initialize an empty list to store extracted coordinates
        coordinates = []

        # Find all <Placemark> elements in the KML file
        for placemark in root.findall(".//kml:Placemark", namespace):
            # Find <coordinates> tags within each <Placemark>
            for coord in placemark.findall(".//kml:coordinates", namespace):
                coords_text = coord.text.strip()  # Extract and clean text from <coordinates> tag
                coord_list = coords_text.split()  # Split the text by space for multiple points
                
                # Process each coordinate (longitude, latitude, altitude)
                for single_coord in coord_list:
                    lon, lat, _ = map(float, single_coord.split(","))  # Extract longitude, latitude, altitude
                    coordinates.append((lat, lon))  # Append latitude and longitude as a tuple

        return coordinates  # Return the list of extracted coordinates
    except Exception as e:
        # Print an error message if parsing fails
        print(f"Error parsing KML file: {e}")
        return None

# Extract coordinates from the KML file
coordinates = extract_coordinates_from_kml(kml_file_path)

# Save the extracted coordinates to a CSV file
if coordinates:  # Check if coordinates were successfully extracted
    output_csv_path = os.path.join("C:/Users/subha/Desktop/DEM/Rudraprayag_DEM", "coordinates.csv")
    # Create a DataFrame with the extracted coordinates
    df = pd.DataFrame(coordinates, columns=["Latitude", "Longitude"])
    # Save the DataFrame to a CSV file without the index
    df.to_csv(output_csv_path, index=False)
    print(f"Coordinates extracted and saved to {output_csv_path}")
else:
    # Print a message if no coordinates were found or an error occurred
    print("No coordinates found or an error occurred.")


In [ ]:
import requests  # Library for making HTTP requests
import pandas as pd  # Library for data manipulation and analysis
import time  # Library for adding delays (used to avoid rate limits)

# Load coordinates from the CSV file
coordinates_file = "C:/Users/subha/Desktop/DEM/Rudraprayag_DEM/coordinates.csv"
coordinates_df = pd.read_csv(coordinates_file)  # Read the CSV file into a DataFrame

# List to store the results
results = []

# Function to fetch elevation data from Open Elevation API with retries
def fetch_elevation(lat, lon, retries=3):
    # Construct the API URL with latitude and longitude
    api_url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    for attempt in range(retries):  # Try multiple times in case of failure
        try:
            # Make an HTTP GET request to fetch elevation data
            response = requests.get(api_url, timeout=100)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()  # Parse the response as JSON
            elevation = data["results"][0]["elevation"]  # Extract elevation value
            return elevation  # Return the elevation
        except Exception as e:
            # Print error message for the failed attempt
            print(f"Error fetching elevation for ({lat}, {lon}), Attempt {attempt + 1}/{retries}: {e}")
            if attempt < retries - 1:
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                return None  # Return None if all retries fail

# Loop through each row in the coordinates DataFrame to fetch elevation
for index, row in coordinates_df.iterrows():
    lat = row["Latitude"]  # Extract latitude
    lon = row["Longitude"]  # Extract longitude
    elevation = fetch_elevation(lat, lon)  # Fetch elevation for the current coordinates
    # Append the latitude, longitude, and elevation to the results list
    results.append({"Latitude": lat, "Longitude": lon, "Elevation": elevation})
    # Print the fetched elevation for the current coordinate
    print(f"Fetched elevation for ({lat}, {lon}): {elevation}")
    time.sleep(5)  # Pause for 5 seconds to avoid API rate limits

# Save the results to a new CSV file
output_file = "C:/Users/subha/Desktop/DEM/Rudraprayag_DEM/elevation_data.csv"
df = pd.DataFrame(results)  # Convert results list to a DataFrame
df.to_csv(output_file, index=False)  # Save the DataFrame to a CSV file without the index
print(f"Elevation data saved to {output_file}")


In [ ]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations
import matplotlib.pyplot as plt  # For plotting
import os  # For handling file paths

# Path to the saved CSV file
csv_path = 'C:/Users/subha/Desktop/DEM/Rudraprayag_DEM/elevation_data.csv'

# Import the CSV data
df = pd.read_csv(csv_path)  # Load the elevation data into a DataFrame
print(f"CSV data imported successfully from: {csv_path}")

# Display the first few rows of the DataFrame
print("Data preview:")
print(df.head())  # Preview the first five rows of the data

# Check for missing values in the Elevation column
print(f"Missing values in Elevation: {df['Elevation'].isna().sum()}")

# Remove rows with NaN in Elevation
# Alternatively, missing values can be filled using df['Elevation'].fillna(value)
df = df.dropna(subset=['Elevation'])

# Extract Latitude, Longitude, and Elevation data as numpy arrays for processing
latitudes = df['Latitude'].values
longitudes = df['Longitude'].values
elevations = df['Elevation'].values

# Debug: Check data ranges for sanity checks
print(f"Latitude range: {latitudes.min()} to {latitudes.max()}")
print(f"Longitude range: {longitudes.min()} to {longitudes.max()}")
print(f"Elevation range: {elevations.min()} to {elevations.max()}")

# Define grid for interpolation with a coarser grid spacing
grid_spacing = 0.001  # Defines the resolution of the interpolation grid
grid_lat, grid_lon = np.meshgrid(
    np.arange(latitudes.min(), latitudes.max(), grid_spacing),
    np.arange(longitudes.min(), longitudes.max(), grid_spacing)
)

# IDW Interpolation Function
def inverse_distance_weighting(x, y, z, xi, yi, power=2):
    """
    Perform Inverse Distance Weighting (IDW) interpolation.
    - x, y: Coordinates of known data points.
    - z: Known values at data points.
    - xi, yi: Coordinates where interpolation is required.
    - power: Weighting power (default=2).
    """
    dist = np.sqrt((xi[:, :, None] - x) ** 2 + (yi[:, :, None] - y) ** 2)  # Calculate distances
    dist[dist == 0] = 1e-10  # Avoid division by zero for overlapping points
    weights = 1 / dist ** power  # Calculate weights using inverse distance formula
    z_interp = np.sum(weights * z, axis=2) / np.sum(weights, axis=2)  # Compute weighted average
    return z_interp

# Apply IDW interpolation
grid_elevation = inverse_distance_weighting(
    longitudes, latitudes, elevations, grid_lon, grid_lat, power=2
)

# Debug: Check interpolated grid values (sample output for verification)
print(f"Interpolated grid elevation sample: {grid_elevation[:5, :5]}")

# Plot the interpolated raster surface
plt.figure(figsize=(10, 8))  # Set the plot size
plt.contourf(grid_lon, grid_lat, grid_elevation, cmap='terrain', levels=100)  # Generate filled contours
plt.colorbar(label='Elevation (m)')  # Add a color bar for elevation values
plt.title('Elevation Surface from IDW Interpolation', fontsize=14)  # Title for the plot
plt.xlabel('Longitude', fontsize=12)  # Label for the X-axis
plt.ylabel('Latitude', fontsize=12)  # Label for the Y-axis
plt.grid(True, linestyle='--', alpha=0.5)  # Add a grid with dashed lines for better readability

# Show the plot
plt.show()  # Display the final plot

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

# Assuming grid_lon, grid_lat, and grid_elevation have been computed already
# These variables represent the longitude, latitude, and interpolated elevation grid.

# Define the path to save the output GeoTIFF file
output_tif = 'C:/Users/subha/Desktop/DEM/Rudraprayg_interpolated.tif'

# Define the transform for rasterio to align the grid to geographical coordinates
# - `west`: x-coordinate of the upper-left corner (minimum longitude)
# - `north`: y-coordinate of the upper-left corner (maximum latitude)
# - `xsize` and `ysize`: Grid spacing in longitude and latitude, respectively
transform = from_origin(
    west=grid_lon.min(),  # Minimum longitude for the upper-left corner
    north=grid_lat.max(),  # Maximum latitude for the upper-left corner
    xsize=grid_spacing,  # Grid spacing in longitude
    ysize=grid_spacing   # Grid spacing in latitude
)

# Write the interpolated elevation data to a GeoTIFF file using rasterio
with rasterio.open(
    output_tif, 'w', driver='GTiff',  # Specify write mode and GeoTIFF format
    height=grid_elevation.shape[0],  # Number of rows in the elevation grid
    width=grid_elevation.shape[1],  # Number of columns in the elevation grid
    count=1,  # Number of bands (1 for single-band elevation data)
    dtype='float32',  # Data type for the elevation values
    crs='+proj=latlong',  # Coordinate reference system (WGS84 latitude-longitude)
    transform=transform  # Transformation matrix to align data to coordinates
) as dst:
    dst.write(grid_elevation, 1)  # Write the elevation grid to the first band

# Print a confirmation message
print(f"Interpolated surface saved as: {output_tif}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming grid_lon, grid_lat, and grid_elevation have been computed already
# These variables represent the longitude, latitude, and interpolated elevation grid.

# Create a new figure for the 3D plot
fig = plt.figure(figsize=(12, 10))  # Set the figure size (12 inches by 10 inches)
ax = fig.add_subplot(111, projection='3d')  # Add a 3D subplot to the figure

# Plot the 3D surface
surf = ax.plot_surface(
    grid_lon,         # Longitude grid (X-axis values)
    grid_lat,         # Latitude grid (Y-axis values)
    grid_elevation,   # Elevation values (Z-axis values)
    cmap='terrain',   # Colormap for the surface ('terrain' for topographical appearance)
    edgecolor='k',    # Black edge lines for grid cells
    alpha=0.7         # Transparency of the surface (0 = fully transparent, 1 = fully opaque)
)

# Add a color bar to indicate elevation values
fig.colorbar(
    surf,            # Surface plot object to associate the color bar
    ax=ax,           # Axis to attach the color bar
    shrink=0.5,      # Shrink the size of the color bar
    aspect=5,        # Aspect ratio of the color bar
    label='Elevation (m)'  # Label for the color bar
)

# Set axis labels and plot title
ax.set_xlabel('Longitude')  # Label for the X-axis
ax.set_ylabel('Latitude')   # Label for the Y-axis
ax.set_zlabel('Elevation (m)')  # Label for the Z-axis
ax.set_title('3D Elevation Surface')  # Title of the plot

# Save the plot as a PNG image
image_path = 'C:/Users/subha/Desktop/DEM/3d_elevation_plot_600dpi.png'  # File path for saving the image
plt.savefig(
    image_path,      # Save the figure to the specified path
    format='png',    # File format (PNG)
    dpi=300,         # Resolution of the saved image (300 dots per inch for high quality)
    bbox_inches='tight'  # Remove extra whitespace around the plot
)

# Notify the user that the plot was saved
print(f"Plot saved as image: {image_path}")

# Display the plot interactively
plt.show()  # Show the plot in an interactive window

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Path to the saved CSV file
csv_path = 'C:/Users/subha/Desktop/DEM/Rudraprayag_DEM/elevation_data.csv'

# Step 1: Import the CSV data
df = pd.read_csv(csv_path)
print(f"CSV data imported successfully from: {csv_path}")

# Step 2: Preview the first few rows of the data
print("Data preview:")
print(df.head())

# Step 3: Check for missing values in the Elevation column
print(f"Missing values in Elevation: {df['Elevation'].isna().sum()}")

# Step 4: Remove rows with NaN values in Elevation
df = df.dropna(subset=['Elevation'])

# Step 5: Extract Latitude, Longitude, and Elevation data from the DataFrame
latitudes = df['Latitude'].values
longitudes = df['Longitude'].values
elevations = df['Elevation'].values

# Debug: Check ranges of the data
print(f"Latitude range: {latitudes.min()} to {latitudes.max()}")
print(f"Longitude range: {longitudes.min()} to {longitudes.max()}")
print(f"Elevation range: {elevations.min()} to {elevations.max()}")

# Step 6: Define a grid for interpolation with a coarser grid spacing
grid_spacing = 0.001  # Increase spacing to create a coarser grid
grid_lat, grid_lon = np.meshgrid(
    np.arange(latitudes.min(), latitudes.max(), grid_spacing),  # Grid for Latitude
    np.arange(longitudes.min(), longitudes.max(), grid_spacing)  # Grid for Longitude
)

# Step 7: Define the IDW interpolation function
def inverse_distance_weighting(x, y, z, xi, yi, power=2):
    """
    Perform Inverse Distance Weighting (IDW) interpolation.
    - x, y: Coordinates of known data points.
    - z: Known values at data points.
    - xi, yi: Coordinates where interpolation is required.
    - power: Weighting power (default=2).
    """
    dist = np.sqrt((xi[:, :, None] - x) ** 2 + (yi[:, :, None] - y) ** 2)  # Calculate distances
    dist[dist == 0] = 1e-10  # Avoid division by zero
    weights = 1 / dist ** power  # Calculate weights based on inverse distance
    z_interp = np.sum(weights * z, axis=2) / np.sum(weights, axis=2)  # Weighted sum of elevations
    return z_interp

# Step 8: Perform IDW interpolation
grid_elevation = inverse_distance_weighting(
    longitudes, latitudes, elevations, grid_lon, grid_lat, power=2
)

# Debug: Check a sample of interpolated elevation grid values
print(f"Interpolated grid elevation sample: {grid_elevation[:5, :5]}")

# Step 9: Create a 3D surface plot using Plotly
fig = go.Figure(data=go.Surface(
    z=grid_elevation,  # Interpolated elevation grid
    x=grid_lon,        # Longitude grid
    y=grid_lat,        # Latitude grid
    colorscale='earth',  # Color scale for elevation
    colorbar_title='Elevation (m)'  # Title for the color bar
))

# Step 10: Customize axis labels and layout options
fig.update_layout(
    title='3D Elevation Surface',  # Title of the plot
    scene=dict(
        xaxis_title='Longitude',  # Label for X-axis
        yaxis_title='Latitude',   # Label for Y-axis
        zaxis_title='Elevation (m)'  # Label for Z-axis
    ),
    width=800,  # Width of the plot
    height=800  # Height of the plot
)

# Step 11: Display the 3D plot interactively
fig.show()


In [ ]:
# Save the 3D plot as an interactive HTML file
html_path = 'C:/Users/subha/Desktop/DEM/3d_elevation_plot.html'  # Modify the path as needed
fig.write_html(html_path)

# Notify the user that the plot has been saved
print(f"Plot saved as HTML: {html_path}")